In [2]:
import math
import numpy as np
import matplotlib.pyplot as plt
import os
import pyvista as pv
import plotly.graph_objs as go

output_path = 'C:/Users/allen/OneDrive/Desktop/Work/Scripts/Lidar Simulation/output_008/'
redwood_path_prefix = "redwood_output/"
guiana_path_prefix = "guiana_output/"

radius = 33.3 / 2

In [3]:
# Define a function to convert a string representation of a list to a list
def parse_list_string(list_string):
    return [float(item.strip()) if '.' in item and item != '-' else int(item.strip()) if item != '-' else None for item in list_string.split('-')]

def read_tree_data(tree_file_path):
    tree_data = {}

    with open(tree_file_path, 'r') as file:
        tree_index = None
        tree_values = {}
        for line in file:
            line = line.strip()
            if line.endswith(':'):
                # Start of a new tree index
                if tree_index is not None:
                    tree_data[tree_index] = tree_values
                    tree_values = {}
                tree_index = line[:-1]
            elif line:
                # Parse field and value
                field, value = line.split(':')
                field = field.strip()
                value = value.strip()
                if value.startswith('- '):
                    # If the value is a list
                    value = parse_list_string(value[2:])
                elif '.' in value:
                    # If the value is a float
                    value = float(value)
                else:
                    # If the value is an integer or string
                    try:
                        value = int(value)
                    except ValueError:
                        pass
                tree_values[field] = value

        # Add the last tree data
        if tree_index is not None:
            tree_data[tree_index] = tree_values

        return tree_data

def parse_list_str(value):
    return [float(x.strip()) if '.' in x and any(c.isdigit() for c in x) else int(float(x.strip())) if x.strip().replace('.', '', 1).isdigit() else None for x in value.split('-')]

def read_circle_data(file_path):
    circle_data = {}
    current_circle_name = None
    current_field_name = None
    current_data = None

    with open(file_path, 'r') as file:
        for line in file:
            line = line.strip()

            # Check if the line is the start of a new circle
            if line.startswith("Circle ["):
                current_circle_name = line.rstrip(":")
                circle_data[current_circle_name] = {}
                current_data = circle_data[current_circle_name]
            elif line.endswith(":"):  # Field name line
                current_field_name = line[:-1]  # Extract field name
                current_data[current_field_name] = []
            elif line.startswith("-"):  # List item line
                if current_field_name:  # Ensure current_field_name is set
                    if current_field_name == "tree_indices":
                        current_data[current_field_name].append(int(line.split("-")[-1].strip()))
                    else:
                        current_data[current_field_name].append(float(line.split("-")[-1].strip()))
            elif ":" in line:  # Single value line
                field_name, field_value = map(str.strip, line.split(":", 1))
                current_data[field_name] = float(field_value)

    return circle_data

def output_circle_indices(circle_indices):
    for circle_name, data in circle_indices.items():
        print(f"{circle_name}:")
        for field, value in data.items():
            if isinstance(value, list):
                print(f"  {field}:")
                for v in value:
                    print(f"    - {v}")
            else:
                print(f"  {field}: {value}")

In [4]:
tree_file_path = output_path + redwood_path_prefix + 'tree_data.txt'

print (f'reading in {tree_file_path}')
redwood_tree_data = read_tree_data(tree_file_path)

tree_file_path = output_path + guiana_path_prefix + 'tree_data.txt'

print (f'reading in {tree_file_path}')
guiana_tree_data = read_tree_data(tree_file_path)

circle_file_path = output_path + redwood_path_prefix + 'circle_data.txt'

print (f'reading in {circle_file_path}')
redwood_circle_indices = read_circle_data(circle_file_path)

circle_file_path = output_path + guiana_path_prefix + 'circle_data.txt'

print (f'reading in {circle_file_path}')
guiana_circle_indices = read_circle_data(circle_file_path)

print(redwood_tree_data)
print(guiana_tree_data)
print(redwood_circle_indices)
print(guiana_circle_indices)

reading in C:/Users/allen/OneDrive/Desktop/Work/Scripts/Lidar Simulation/output_008/redwood_output/tree_data.txt
reading in C:/Users/allen/OneDrive/Desktop/Work/Scripts/Lidar Simulation/output_008/guiana_output/tree_data.txt
reading in C:/Users/allen/OneDrive/Desktop/Work/Scripts/Lidar Simulation/output_008/redwood_output/circle_data.txt
reading in C:/Users/allen/OneDrive/Desktop/Work/Scripts/Lidar Simulation/output_008/guiana_output/circle_data.txt
{'Tree [1]': {'crown_base': 6.53279, 'crown_d1': 4.75503, 'crown_d2': 2.70483, 'crown_d3': 9.46251, 'crown_center_height': 11.26404, 'tree_height': 16.01908, 'crown_base_height': 6.50901, 'crown_volume': 145.72089, 'single_leaf_area': 6e-05, 'total_leaf_area': 21.85813, 'total_tree_branch_area': 509.10734, 'total_tree_stem_area': 0, 'total_tree_ground_area': 71.03238, 'total_tree_canopy_veg_area': 530.96547, 'x_coord': -58.22, 'y_coord': 39.89}, 'Tree [2]': {'crown_base': 7.02676, 'crown_d1': 5.83397, 'crown_d2': 3.09267, 'crown_d3': 11.609

In [5]:
def plot_trees_and_circles_3d(tree_data, circle_indices, region):
    # Create a PyVista mesh for trees
    tree_meshes = []
    
    # Set opacity for trees and circles
    tree_opacity = 0.2
    circle_opacity = 0.2

    for tree_index, tree_info in tree_data.items():
        x = tree_info.get('x_coord')
        y = tree_info.get('y_coord')
        d1 = tree_info.get('crown_d1')
        d2 = tree_info.get('crown_d2')
        tree_height = tree_info.get('tree_height')
        tree = pv.Cylinder(center=(x, y, tree_height / 2), direction=(0, 0, 1), radius=d2, height=tree_height)
        tree_meshes.append(tree)

    # Create a PyVista mesh for circles
    circle_meshes = []
    for circle_name, data in circle_indices.items():
        # Remove the prefix "Circle" and strip whitespace
        stripped_name = circle_name.lstrip('Circle').strip()
        i, j = map(int, stripped_name.strip('[]').split(','))
        height = 50  # Define a small height for the cylinder
        center_x = (j - 2) * 2 * radius
        center_y = (i - 2) * 2 * radius
        circle = pv.Cylinder(center=(center_x, center_y, 25), direction=(0, 0, 1), radius=radius, height=height)
        circle_meshes.append(circle)

    # Combine all tree and circle meshes into a single PyVista mesh
    all_meshes = tree_meshes + circle_meshes

    traces = []
    for mesh in all_meshes:
        vertices = np.array(mesh.points)
        faces = np.array(mesh.faces)
        
        # Reshape faces array to split it into groups of three indices
        faces = faces.reshape(-1, 3)
        
        # Extract face indices for each triangle
        i, j, k = faces[:, 0], faces[:, 1], faces[:, 2]

        # Create a trace for the mesh
        trace = go.Mesh3d(
            x=vertices[:, 0],
            y=vertices[:, 1],
            z=vertices[:, 2],
            i=i,
            j=j,
            k=k,
            opacity=0.3,  # Set opacity for transparency
            color='blue',
        )
        traces.append(trace)

    # Set layout and title
    layout = go.Layout(
        title=region,
        scene=dict(
            xaxis=dict(title='X'),
            yaxis=dict(title='Y'),
            zaxis=dict(title='Z'),
        )
    )

    # Create Plotly figure
    fig = go.Figure(data=traces, layout=layout)

    # Save the plot to an HTML file
    filename = f'{region.replace(" ", "_")}.html'
    output_file_path = os.path.join(output_path, filename)
    fig.write_html(output_file_path)

    # Show the plot using PyVista
    p = pv.Plotter(notebook=False)
    for mesh in all_meshes:
        if mesh in tree_meshes:
            p.add_mesh(mesh, color='green', opacity=tree_opacity)
        else:
            p.add_mesh(mesh, color='red', opacity=circle_opacity)
    p.add_text(region, font_size=40, color='black', position='upper_edge')
    p.show()

In [6]:
# Plot for Redwood
plot_trees_and_circles_3d(redwood_tree_data, redwood_circle_indices, 'California Redwoods')

# # Plot for French Guiana
# plot_trees_and_circles_3d(guiana_tree_data, guiana_circle_indices, 'French Guiana')